Lo primero que tendermos que realizar es importar la base de datos y verificar si esta tiene huecos dentro de ella.

In [3]:
import pandas as pd

df=pd.read_csv('Khan.csv')
missing_values = df.isnull().sum().sum()
print("Valores faltantes:")
print(missing_values)
print("Valores originales en el conjunto de datos:")
print(df.shape)

Valores faltantes:
0
Valores originales en el conjunto de datos:
(83, 2309)


Podemos observar que la base de datos no cuenta con ningun hueco.

In [4]:
import numpy as np


# Supongamos que tu DataFrame se llama df y tiene una columna 'y' que indica las clases
# y las demás columnas representan los genes.

# Filtrar las observaciones de las clases 2 y 4
clase_2 = df[df['y'] == 2]
clase_4 = df[df['y'] == 4]

# Calcular la diferencia de promedios para cada gen
# Excluimos la columna 'y' al calcular los promedios
diferencias = (clase_2.mean() - clase_4.mean()).drop('y')

# Ordenar las diferencias en orden descendente
diferencias_ordenadas = diferencias.abs().sort_values(ascending=False)

# Imprimir los 10 genes con la mayor diferencia de medias
print("Los 10 genes con la mayor diferencia de medias entre las clases 2 y 4:")
print(diferencias_ordenadas.head(10))

Los 10 genes con la mayor diferencia de medias entre las clases 2 y 4:
X187     3.323151
X509     2.906537
X2046    2.424515
X2050    2.401783
X129     2.165185
X1645    2.065460
X1319    2.045941
X1955    2.037340
X1003    2.011337
X246     1.837830
dtype: float64


Acabamos de hacer una seleccion de caracteristicas comparando los promedios entre la clase 2 y la clase 4 de nuestro dataframe pero para tener una mejor seleccion compararemos sus p-values y estadisticos t.

In [5]:
from scipy.stats import ttest_ind
from statsmodels.stats.multitest import multipletests
genes = df.columns.drop('y')  # Excluir la columna de clase
t_values = []
p_values = []

# Calcular el estadístico t y el p-value para cada gen
for gen in genes:
    t_stat, p_val = ttest_ind(clase_2[gen], clase_4[gen], equal_var=False)  # Prueba t de Student
    t_values.append(t_stat)
    p_values.append(p_val)

# Convertir los resultados a un DataFrame
results = pd.DataFrame({
    'Gen': genes,
    't_value': t_values,
    'p_value': p_values
})

# Aplicar correcciones por múltiples pruebas
alpha = 0.05

# Bonferroni
results['Bonferroni_corrected'], results['Bonferroni_pval'], _, _ = multipletests(results['p_value'], alpha=alpha, method='bonferroni')

# Holm
results['Holm_corrected'], results['Holm_pval'], _, _ = multipletests(results['p_value'], alpha=alpha, method='holm')

# Benjamini-Hochberg
results['BH_corrected'], results['BH_pval'], _, _ = multipletests(results['p_value'], alpha=alpha, method='fdr_bh')

# Filtrar genes con diferencias significativas para cada método
significant_bonferroni = results[results['Bonferroni_corrected']]
significant_holm = results[results['Holm_corrected']]
significant_bh = results[results['BH_corrected']]

sorted_bonferroni = results.sort_values(by='Bonferroni_pval')
sorted_holm = results.sort_values(by='Holm_pval')
sorted_bh = results.sort_values(by='BH_pval')

# Imprimir los resultados
print("Genes con diferencias significativas (Bonferroni):")
print(sorted_bonferroni)



Genes con diferencias significativas (Bonferroni):
        Gen    t_value       p_value  Bonferroni_corrected  Bonferroni_pval  \
1002  X1003 -12.879590  4.998692e-17                  True     1.153698e-13   
186    X187 -12.229464  3.716887e-16                  True     8.578576e-13   
2049  X2050 -10.983249  4.084836e-15                  True     9.427801e-12   
1954  X1955 -12.579058  5.307128e-15                  True     1.224885e-11   
1644  X1645  10.827754  8.262889e-15                  True     1.907075e-11   
...     ...        ...           ...                   ...              ...   
16      X17   1.720567  9.249593e-02                 False     1.000000e+00   
2305  X2306  -1.463430  1.520007e-01                 False     1.000000e+00   
2306  X2307  -1.529576  1.322255e-01                 False     1.000000e+00   
2307  X2308   0.989658  3.269311e-01                 False     1.000000e+00   
0        X1   0.739831  4.627656e-01                 False     1.000000e+00   



In [6]:
print("Genes con diferencias significativas (Holm):")
print(sorted_holm)


Genes con diferencias significativas (Holm):
        Gen    t_value       p_value  Bonferroni_corrected  Bonferroni_pval  \
1002  X1003 -12.879590  4.998692e-17                  True     1.153698e-13   
186    X187 -12.229464  3.716887e-16                  True     8.578576e-13   
2049  X2050 -10.983249  4.084836e-15                  True     9.427801e-12   
1954  X1955 -12.579058  5.307128e-15                  True     1.224885e-11   
1644  X1645  10.827754  8.262889e-15                  True     1.907075e-11   
...     ...        ...           ...                   ...              ...   
16      X17   1.720567  9.249593e-02                 False     1.000000e+00   
2305  X2306  -1.463430  1.520007e-01                 False     1.000000e+00   
2306  X2307  -1.529576  1.322255e-01                 False     1.000000e+00   
2307  X2308   0.989658  3.269311e-01                 False     1.000000e+00   
0        X1   0.739831  4.627656e-01                 False     1.000000e+00   

      

In [7]:
print("Genes con diferencias significativas (Benjamini-Hochberg):")
print(sorted_bh)

Genes con diferencias significativas (Benjamini-Hochberg):
        Gen    t_value       p_value  Bonferroni_corrected  Bonferroni_pval  \
1002  X1003 -12.879590  4.998692e-17                  True     1.153698e-13   
186    X187 -12.229464  3.716887e-16                  True     8.578576e-13   
1954  X1955 -12.579058  5.307128e-15                  True     1.224885e-11   
2049  X2050 -10.983249  4.084836e-15                  True     9.427801e-12   
1644  X1645  10.827754  8.262889e-15                  True     1.907075e-11   
...     ...        ...           ...                   ...              ...   
1962  X1963  -0.005855  9.953544e-01                 False     1.000000e+00   
446    X447  -0.006869  9.945509e-01                 False     1.000000e+00   
582    X583  -0.004050  9.967843e-01                 False     1.000000e+00   
568    X569   0.003052  9.975796e-01                 False     1.000000e+00   
1167  X1168   0.000063  9.999498e-01                 False     1.000000e

Podemos observar los diferentes p-values de los datos organisados de mas significativo a menos significativo.

Ahora realizaremos un analisis de variansa entre los 4 casos posibles de la salida y con eso realisaremos nuestra seleccion final de caracteristicas tomando en cuenta que descartaremos todos los que esten abajo de 0.05 en su p-value.

In [8]:
from scipy.stats import f_oneway

# Estratificar los datos por clase
clases = [df[df['y'] == clase] for clase in df['y'].unique()]

# Excluir la columna 'y' y preparar los datos para ANOVA
genes = df.columns.drop('y')
anova_results = []

for gen in genes:
    # Extraer los valores del gen para cada clase
    valores_por_clase = [clase[gen].values for clase in clases]
    
    # Realizar la prueba ANOVA
    f_stat, p_val = f_oneway(*valores_por_clase)
    anova_results.append({'Gen': gen, 'F_stat': f_stat, 'p_value': p_val})

# Convertir los resultados a un DataFrame
anova_df = pd.DataFrame(anova_results)

# Ordenar por p-value
anova_df_sorted = anova_df.sort_values(by='p_value')

# Imprimir los resultados ordenados
print("Resultados de ANOVA ordenados por p-value:")
print(anova_df_sorted)

Resultados de ANOVA ordenados por p-value:
        Gen     F_stat       p_value
1954  X1955  84.364086  1.459035e-24
1388  X1389  83.817537  1.772751e-24
1002  X1003  77.795622  1.618988e-23
2049  X2050  69.230799  4.733702e-22
245    X246  68.414042  6.633722e-22
...     ...        ...           ...
769    X770   0.065309  9.780858e-01
1139  X1140   0.054007  9.833477e-01
440    X441   0.045939  9.868383e-01
937    X938   0.036097  9.907486e-01
669    X670   0.010611  9.984902e-01

[2308 rows x 3 columns]


prosedamos a hacer el seleccion de caractaristicas

In [9]:
# Filtrar genes con p-value menor o igual a 0.05
selected_features = anova_df_sorted[anova_df_sorted['p_value'] <= 0.05]

# Imprimir las características seleccionadas
print("Características seleccionadas (p-value <= 0.05):")
print(selected_features)

Características seleccionadas (p-value <= 0.05):
        Gen     F_stat       p_value
1954  X1955  84.364086  1.459035e-24
1388  X1389  83.817537  1.772751e-24
1002  X1003  77.795622  1.618988e-23
2049  X2050  69.230799  4.733702e-22
245    X246  68.414042  6.633722e-22
...     ...        ...           ...
1337  X1338   2.734705  4.912248e-02
948    X949   2.732694  4.924375e-02
658    X659   2.732400  4.926153e-02
1033  X1034   2.723469  4.980396e-02
1221  X1222   2.720638  4.997709e-02

[1303 rows x 3 columns]


Ahora separemos todo nuestro nuevo conjunto en train y test.

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(df.drop('y', axis=1), df.y, test_size=0.2)

Ya que separamos los datos generaremos 3 modelos SVM uno LINEAL, otro POLINOMIAL de tercer orden y uno RADIAL.

In [13]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report

# Crear y entrenar el modelo SVM con kernel lineal
svm_linear = SVC(kernel='linear')
svm_linear.fit(X_train, Y_train)


# Crear y entrenar el modelo SVM con kernel polinomial de orden 3
svm_poly = SVC(kernel='poly', degree=3)
svm_poly.fit(X_train, Y_train)


# Crear y entrenar el modelo SVM con kernel radial
svm_rbf = SVC(kernel='rbf')
svm_rbf.fit(X_train, Y_train)


# Evaluar los modelos en el conjunto de prueba
print("\nEvaluación del modelo SVM con kernel lineal:")
print(classification_report(Y_test, svm_linear.predict(X_test)))



Evaluación del modelo SVM con kernel lineal:
              precision    recall  f1-score   support

           1       1.00      1.00      1.00         4
           2       1.00      1.00      1.00         4
           3       1.00      1.00      1.00         3
           4       1.00      1.00      1.00         6

    accuracy                           1.00        17
   macro avg       1.00      1.00      1.00        17
weighted avg       1.00      1.00      1.00        17



In [14]:

print("\nEvaluación del modelo SVM con kernel polinomial de orden 3:")
print(classification_report(Y_test, svm_poly.predict(X_test)))


Evaluación del modelo SVM con kernel polinomial de orden 3:
              precision    recall  f1-score   support

           1       1.00      1.00      1.00         4
           2       1.00      1.00      1.00         4
           3       1.00      1.00      1.00         3
           4       1.00      1.00      1.00         6

    accuracy                           1.00        17
   macro avg       1.00      1.00      1.00        17
weighted avg       1.00      1.00      1.00        17



In [15]:

print("\nEvaluación del modelo SVM con kernel radial:")
print(classification_report(Y_test, svm_rbf.predict(X_test)))


Evaluación del modelo SVM con kernel radial:
              precision    recall  f1-score   support

           1       1.00      0.75      0.86         4
           2       0.80      1.00      0.89         4
           3       1.00      1.00      1.00         3
           4       1.00      1.00      1.00         6

    accuracy                           0.94        17
   macro avg       0.95      0.94      0.94        17
weighted avg       0.95      0.94      0.94        17



Parece ser que nuestros modelos Lineal y Polinomial pueden predecir con un 100% de certesa cual de las 4 clases seria el nuevo valor, este solo reduciendose en el analisis Radial a un 94%.

In [16]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Predicciones de los modelos
y_pred_linear = svm_linear.predict(X_test)
y_pred_poly = svm_poly.predict(X_test)
y_pred_rbf = svm_rbf.predict(X_test)

# Función para calcular métricas
def calcular_metricas(y_true, y_pred, modelo):
    print(f"Métricas para el modelo {modelo}:")
    print(f"Precisión: {accuracy_score(y_true, y_pred):.4f}")
    print(f"Exactitud: {precision_score(y_true, y_pred, average='weighted'):.4f}")
    print(f"Sensibilidad (Recall): {recall_score(y_true, y_pred, average='weighted'):.4f}")
    print(f"Puntuación F1: {f1_score(y_true, y_pred, average='weighted'):.4f}")
    print("Matriz de confusión:")
    print(confusion_matrix(y_true, y_pred))
    print("\n")

# Calcular métricas para cada modelo
calcular_metricas(Y_test, y_pred_linear, "SVM Lineal")
calcular_metricas(Y_test, y_pred_poly, "SVM Polinomial")
calcular_metricas(Y_test, y_pred_rbf, "SVM Radial")

Métricas para el modelo SVM Lineal:
Precisión: 1.0000
Exactitud: 1.0000
Sensibilidad (Recall): 1.0000
Puntuación F1: 1.0000
Matriz de confusión:
[[4 0 0 0]
 [0 4 0 0]
 [0 0 3 0]
 [0 0 0 6]]


Métricas para el modelo SVM Polinomial:
Precisión: 1.0000
Exactitud: 1.0000
Sensibilidad (Recall): 1.0000
Puntuación F1: 1.0000
Matriz de confusión:
[[4 0 0 0]
 [0 4 0 0]
 [0 0 3 0]
 [0 0 0 6]]


Métricas para el modelo SVM Radial:
Precisión: 0.9412
Exactitud: 0.9529
Sensibilidad (Recall): 0.9412
Puntuación F1: 0.9402
Matriz de confusión:
[[3 1 0 0]
 [0 4 0 0]
 [0 0 3 0]
 [0 0 0 6]]




Presentado los modelos de una forma que sea mas facil de comparar podemos ver que el lineal y polinomial serian los mas adecuados para nuestro analisis ya que al parecer no parecen cometer errores.